In [2]:
import os
import pandas as pd
import csv
import matplotlib.pyplot as plt
import networkx as nx
import pickle
import numpy as np
import matplotlib as mpl
import seaborn as sns 
import json
from networkx.drawing.nx_agraph import graphviz_layout
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
from shapely.geometry import Polygon
from keplergl import KeplerGl
from collections import Counter

In [3]:
path_harvey = '/Users/liqingchun/Google Drive/Dissertation/Dataset/Safegraph Data/SafeGraph Raw Data/'
path_network = '/Users/liqingchun/Google Drive/Dissertation/Dataset/Safegraph Data/SafeGraph Raw Data/OD_network/'
path_core = '/Users/liqingchun/Google Drive/Dissertation/Dataset/Safegraph Data/Safegraph Core data/CorePlacesMay2020Release-CORE_POI-2020_04-2020-05-06/'
path_census = '/Users/liqingchun/Google Drive/Dissertation/Dataset/Safegraph Data/safegraph_open_census_data/metadata/'

In [4]:
os.chdir(path_census)
with open('census_geo.pkl', 'rb') as file:
    census = pickle.load(file)

In [5]:
os.chdir(path_core)
with open('core_pickle.pkl', 'rb') as file:
    core = pickle.load(file)

In [14]:
def Hotspots_category(file, name, path=path_network):
    os.chdir(path)
    with open(file, 'rb') as file:
        safegraph_id = pickle.load(file)
        
    safegraph_id = safegraph_id.drop(['cbg_latitude', 'cbg_longitude'], axis=1).merge(census, on=['cbg'])
    safegraph_id = safegraph_id.merge(core[['safegraph_place_id', 'top_category']], on=['safegraph_place_id'])
    safegraph_id.to_csv(name+'_network_information.csv')
    hotspots_category_list = list(safegraph_id[safegraph_id.hotspot_destination == True].top_category)
        
    cate_dict = Counter(hotspots_category_list)
    top_5 = sorted(cate_dict.items(), key=lambda kv: kv[1], reverse=True)
    
    with open(name+'.csv', 'w', newline='') as csvfile:
        list_writer = csv.writer(csvfile)
        for each in top_5:
            list_writer.writerow(each)

In [82]:
Hotspots_category('2020-04-27-san jose-cbg-home-poi-visits-network.pkl', 'san_jose_0427', path=path_network+'san jose/dataframes/')

In [43]:
def Hotspots_visit_category(file, name, path=path_network):
    os.chdir(path)
    with open(file, 'rb') as file:
        safegraph_id = pickle.load(file)
        
    safegraph_id = safegraph_id.merge(core[['safegraph_place_id', 'top_category']], on=['safegraph_place_id'])
    safegraph_id = safegraph_id[safegraph_id.hotspot_destination == True]
    category_visit = safegraph_id.groupby('top_category').sum().sort_values("visits",inplace=False, ascending=False)
    category_visit = category_visit[['visits']]
    category_visit.to_csv(name+'_visits.csv', header=False)
  

In [114]:
Hotspots_visit_category('2020-04-27-san jose-cbg-home-poi-visits-network.pkl', 'san jose_0427', path=path_network+'san jose/dataframes/')